In [1]:
'''
RNN model for play generation
'''

'\nRNN model for play generation\n'

In [2]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [4]:
print(path_to_file)

/root/.keras/datasets/shakespeare.txt


In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('length of text :{} characters'.format(len(text)))

length of text :1115394 characters


In [6]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
'''
Encoding the text to pass into training our model
'''
vocab = sorted(set(text))

char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
print('text: ', text[:13])
print('Encoded: ', text_to_int(text[:13]))

text:  First Citizen
Encoded:  [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
'''
Decoding function to covert from integer to text
'''

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [10]:
'''
for this model to predict future characters or future scenes of play, we need to set length of the sequence, this will be our input
and then we move on step towards the right and set that words at the output

length = 4
Hell - input, ello-output

'''

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)

In [12]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return  input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for x,y in dataset.take(2):
  print('\n\nExample\n')
  print('input')
  print(int_to_text(x))
  print('\noutput')
  print(int_to_text(y))



Example

input
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

output
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


Example

input
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

output
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [14]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(
          vocab_size, embedding_dim, batch_input_shape = [batch_size, None]),
      tf.keras.layers.LSTM(rnn_units,
                           return_sequences = True,
                           stateful = True,
                          recurrent_initializer = 'glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
      ])
  return model


model = build_model(VOCAB_SIZE,EMBEDDING_DIM,RNN_UNITS,BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
'''
creating our loss fucntion because tf doesn't have any loss function to compute how is our model performing,
our model produces, 3 dimensional output
'''

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [23]:
model.compile(optimizer='adam',loss = loss)

In [25]:
history = model.fit(data, epochs=10,callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 14s 70ms/step - loss: 1.2767
Epoch 2/10
172/172 [==============================] - 14s 72ms/step - loss: 1.2522
Epoch 3/10
172/172 [==============================] - 14s 73ms/step - loss: 1.2285
Epoch 4/10
172/172 [==============================] - 15s 76ms/step - loss: 1.2061
Epoch 5/10
172/172 [==============================] - 15s 74ms/step - loss: 1.1818
Epoch 6/10
172/172 [==============================] - 15s 77ms/step - loss: 1.1571
Epoch 7/10
172/172 [==============================] - 15s 78ms/step - loss: 1.1311
Epoch 8/10
172/172 [==============================] - 15s 80ms/step - loss: 1.1040
Epoch 9/10
172/172 [==============================] - 15s 79ms/step - loss: 1.0770
Epoch 10/10
172/172 [==============================] - 15s 80ms/step - loss: 1.0475


In [26]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [27]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [29]:
checkpoint_num = 5
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: ignored

In [30]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [31]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: yo
your sight;
Thy tears are rip. Show me still answer him?

SICINIUS:
Yes, you must welcome the Parlica.

All:
Speak I will bear it as ge to have his wreck.

CATESBY:
Verd not, methinks, peace is so: dies thou nd the generomest hivery.

QUEEN MARGARET:
Why all, above my Juliet,
And smoots the branch of his piercing boy.

SIR STEPHEN SCROOP:
So soon that an usuty is Baptia there?

BUSHY:
Come, sir, now, sicre-cear'd than these blood
Full of no farthest service.

ESCALUS:
Your give your majesty, sir, if I should not
ved a king, Camillo, boling rough,
And son degenation, fair before have attempt
the buried; on nothing give me lastfully; feast, bre Pling of Bantugement,
He makes her card thy care: O, that you had not,
With all the suddener of your mind-dately dead!

QUEEN MARGARET:
Those violent m
